## Introduction and Setup**
This notebook implements a FAIR based cyber risk quantification method:

_Risk = Loss Event Frequency * Loss Magnitude_

**_Read more here: [substack article URL]_**

## Instructive Model

### Set Variables

In [52]:
# MODEL INPUTS - Set variables here to run the model
COMPANIES_IN_SCOPE = 7
YEARS_OBSERVED = 4
LOSSES_OBSERVED = 0          # losses among the 7 custodians during 2020–1H25

BTC_PRICE = 108_136.09       # USD per BTC; Computed value is based on price per BTC as of 10:06 PM ET on 08/31/2025
LM_PCT = 0.70                # % of exposed value lost if a breach succeeds (e.g., 30%)
USE_UPPER_BOUND = False      # True = conservative LEF (rule-of-three), False = point estimate

### Set Example Scenarios

In [58]:
# Two example companies (feel free to edit/add):
scenarios = [
    {"name": "Incline — concentrated", "btc_holdings": 819, "platforms": 1},  # all on one platform; based on AirNet Technology Inc (ANTE) holdings
    {"name": "Incline — diversified",  "btc_holdings": 819,  "platforms": 4},  # split across 4 platforms
]

### Loss Event Frequency (LEF) Estimation

We treat each **company-year** as one unit of exposure (one company at risk for one year).

- **Exposure basis:**  
  `company_years = companies × years`

- **LEF (base-case, small-sample friendly — Laplace / rule of succession):**  
  `LEF_laplace = (losses + 1) / (company_years + 2)`

- **Alternative point estimate (slightly less conservative: Jeffreys Prior ):**  
  `LEF_jeffreys = (losses + 0.5) / (company_years + 1)`

- **Conservative 95% upper bound (for zero losses, “rule of three”):**  
  `LEF_upper95 ≈ 3 / company_years`  *(use as a stress-test ceiling, not as the base number)*

**Why this:** These small-sample estimators avoid collapsing risk to 0 when you observed 0 losses, while staying simple. Use **Laplace** for your everyday/base estimate and show the **rule-of-three** as a conservative ceiling to communicate uncertainty.


In [59]:
# === LEF via Laplace (rule of succession) ===
COMPANIES_IN_SCOPE = 7
YEARS_OBSERVED = 5
LOSSES_OBSERVED = 0  # observed loss events in the 7-custodian cohort (2022–1H25)

company_years = COMPANIES_IN_SCOPE * YEARS_OBSERVED  # exposure denominator

# Point estimates
lef_laplace  = (LOSSES_OBSERVED + 1) / (company_years + 2)
lef_jeffreys = (LOSSES_OBSERVED + 0.5) / (company_years + 1)

# 95% upper bound for k=0 (not a point estimate)
lef_upper95 = (3 / company_years) if (LOSSES_OBSERVED == 0 and company_years > 0) else None

# Use Laplace as the model LEF
LEF = lef_laplace

print("LEF Estimation (Wallet/Key-Compromise @ Top-Tier Custodians)")
print("=" * 72)
print(f"Exposure basis: {COMPANIES_IN_SCOPE} companies × {YEARS_OBSERVED} years = {company_years} company-years")
print(f"Observed loss events: {LOSSES_OBSERVED}")
print(f"LEF (Laplace, point):   {lef_laplace:.2%}   <-- used")
print(f"LEF (Jeffreys, point):  {lef_jeffreys:.2%}")
if lef_upper95 is not None:
    print(f"LEF (rule-of-three 95% upper): {lef_upper95:.2%}")



LEF Estimation (Wallet/Key-Compromise @ Top-Tier Custodians)
Exposure basis: 7 companies × 5 years = 35 company-years
Observed loss events: 0
LEF (Laplace, point):   2.70%   <-- used
LEF (Jeffreys, point):  1.39%
LEF (rule-of-three 95% upper): 8.57%


### Scenario Build/Setup
Each scenario defines:
    - btc_holdings: total BTC held by the company
    - platforms: how many platforms/custodians hold those BTC (assume even split)
    - Concentration factor = 1 / platforms (blast radius shrinks if you spread holdings)

In [61]:
# === Auto-scaling money formatter (K/M/B/T) ===
def fmt_money(x, unit="auto", decimals=None):
    """
    Format USD with thousands separators and a scale suffix.
    unit: "auto", "", "K", "M", "B", or "T"
    decimals: None -> 0 for ''/K, 2 for M/B/T; or pass an int to override.
    """
    scales = [("", 1), ("K", 1_000), ("M", 1_000_000), ("B", 1_000_000_000), ("T", 1_000_000_000_000)]

    if unit == "auto":
        # choose the largest scale whose divisor is <= |x|
        absx = abs(x)
        suf, div = "", 1
        for s, d in scales:
            if absx >= d:
                suf, div = s, d
        # default decimals: 0 for ''/K, 2 for M/B/T
        if decimals is None:
            decimals = 0 if suf in ("", "K") else 2
        return f"${x/div:,.{decimals}f}{suf}"
    else:
        # fixed unit
        lookup = dict(scales)
        div = lookup[unit]
        if decimals is None:
            decimals = 0 if unit in ("", "K") else 2
        return f"${x/div:,.{decimals}f}{unit}"

# assumes LEF, LM_PCT, BTC_PRICE, and 'scenarios' are defined earlier
def pct(x): return f"{100*x:.2f}%"

SHOW_MULTIYEAR = True
HORIZON_YEARS = 5

print("\nRisk Summary per Company (Base LEF)")
print("=" * 70)
print(f"Annual breach probability (LEF): {LEF:.2%}  |  Note: EAL = LEF × LM (long-run average)\n")

for s in scenarios:
    name = s["name"]
    btc = float(s["btc_holdings"])
    platforms = max(1, int(s.get("platforms", 1)))
    conc = 1 / platforms

    company_value = btc * BTC_PRICE
    LM = LM_PCT * company_value * conc              # loss if a breach occurs (one event)
    EAL = LEF * LM                                  # mean loss per year (long-run average)
    p1y = LEF
    pHy = 1 - (1 - LEF) ** HORIZON_YEARS            # multi-year chance of ≥1 breach

    print(name)
    print("-" * len(name))
    print(f"Holdings: {btc:,.0f} BTC  |  Platforms: {platforms}  |  Concentration: {conc:.2f}")
    print("Probability")
    print(f"  1-year breach probability (LEF):            {pct(p1y)}")
    if SHOW_MULTIYEAR:
        print(f"  {HORIZON_YEARS}-year chance of ≥1 breach:      {pct(pHy)}")
    print("Impact if it happens")
    print(f"  Loss if a breach occurs (LM, one event):    {fmt_money(LM)}")
    print("Risk-weighted average")
    print(f"  Mean loss per year (EAL = LEF × LM):        {fmt_money(EAL)}")
    print("  Note: EAL is a long-run average; most years $0, rare year near LM.\n")



Risk Summary per Company (Base LEF)
Annual breach probability (LEF): 2.70%  |  Note: EAL = LEF × LM (long-run average)

Incline — concentrated
----------------------
Holdings: 819 BTC  |  Platforms: 1  |  Concentration: 1.00
Probability
  1-year breach probability (LEF):            2.70%
  5-year chance of ≥1 breach:      12.80%
Impact if it happens
  Loss if a breach occurs (LM, one event):    $61.99M
Risk-weighted average
  Mean loss per year (EAL = LEF × LM):        $1.68M
  Note: EAL is a long-run average; most years $0, rare year near LM.

Incline — diversified
---------------------
Holdings: 819 BTC  |  Platforms: 4  |  Concentration: 0.25
Probability
  1-year breach probability (LEF):            2.70%
  5-year chance of ≥1 breach:      12.80%
Impact if it happens
  Loss if a breach occurs (LM, one event):    $15.50M
Risk-weighted average
  Mean loss per year (EAL = LEF × LM):        $419K
  Note: EAL is a long-run average; most years $0, rare year near LM.

